In [1]:
import numpy as np
import pyci
import psi4

In [2]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.AOint('aug-cc-pvdz', 'He.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('mo_scf_info.npz')['eps_a']
Ca = np.load('mo_scf_info.npz')['Ca'] 
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints)
del ao_erints

Size of the ERI tensor will be 0.00 Gb

Ground state SCF Energy : -2.85570467 

Nuclear repulsion energy : 0.00000000 

Total electronic energy : -2.85570467 



In [3]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 9  
Molecular orbitals: 9  
Spin Orbitals     : 18  
Alpha Orbitals    : 1  
Beta Orbitals     : 1  
Occupied Orbitals : 1  
Virtual Orbitals  : 8  



In [7]:
orbinfo = (2, nmo)
active_space = (1,8)
singles = True
full_cis = True
doubles = False
options = singles, full_cis, doubles
scf_energy = aoint.scf_energy
HCISD = pyci.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=8)

In [8]:
vals, vecs = np.linalg.eigh(HCISD)

In [11]:
vals[0], psi4.energy('CISD')


(-2.855704667710429, -2.8557046677104294)